In [33]:
import pandas as pd
import datetime as dt
import json
import os
import time
import math
import numpy as np
import talib
from technical import *

In [34]:
rtdlist = [ f for f in os.listdir('../rtd/') if f.endswith(".json") ]
ohlclist = [ f for f in os.listdir('../ohlc/') if f.endswith(".csv") ]
def telegram(msg):
    import requests
    print(msg)
    url="https://api.telegram.org/bot1362017119:qwertyuiop"+str(msg)
    #requests.get(url)

In [35]:
book={'Time':[],'id':[],'Pair':[],'Side':[],'Qty':[], 'Trail':[],'TP':[],'SL':[],'PL':[],'OLTP':[],'CLTP':[],'CTime':[]}
pairs=['EUR/USD', 'GBP/USD', 'USD/CHF','AUD/USD']
stopped_pair={'EUR/USD':False, 'GBP/USD':False, 'USD/CHF':False,'AUD/USD':False}
maxtrade=10000
max_spread_percent=0.00018
demo=True
base_qty = 75
is_mis=True
bookpath='./book.csv'
trade_duration=60*60*4
waiting_time=60*60*4
max_concurrent_trade=4
max_profit_day=60
max_loss_day=-80
max_loss_per_trade=-5.2
suspended_pair=[]
max_open_trade_per_pair=1
max_concurrent_pair=1
suspension_limit=-5
max_profit_per_day_per_pair=4
suspnsion_time=20*60*60
atr_value=0.5
exp_day_next=False
trade_stop=False
def dynamic_bookpath():
    path= './EURUSD-OH-OL-COMPARE-4HR-CANDLE-15X.csv'
    return path

In [36]:
def modification_date():
    t = os.path.getmtime('../rtd/EURUSD.json')
    return dt.datetime.fromtimestamp(t)>dt.datetime.now()-dt.timedelta(seconds=3600)
def tradeTime(offset=False,starttime=None,endtime=None):
    time_now=dt.datetime.now().hour*100+dt.datetime.now().minute
    if offset==True:
        if not endtime>time_now>starttime:
            return False
    if modification_date() ==  False:
        return False
    if dt.date.today().weekday() in [5,6]:
        return False
    return True

In [37]:
def convert_ohlc(data,time='240Min'):
    try:
        data.index=data.date
    except:
        pass
    ohlc_dict = {                                                                                                             
        'Open':'first',                                                                                                    
        'High':'max',                                                                                                       
        'Low':'min',                                                                                                        
        'Close': 'last',                                                                                                    
    }
    return data.resample(time, closed='right').agg(ohlc_dict).dropna(how='any')

def ohlc(pair='EURUSD',tme='240Min'):
    for i in range(20):
        try:
            pair=pair.replace('/','')
            data=pd.read_csv('../ohlc/'+pair+'.csv')
            data.date=pd.to_datetime(data.date)
#             return data
            return convert_ohlc(data,time=tme)
        except:
            time.sleep(0.2)
    return    

def rtd(pair='EURUSD'):
    for i in range(20):
        try:
            pair=pair.replace('/','')
            with open('../rtd/'+pair+'.json', 'r') as fp:
                data= json.load(fp)
            data['ltp']=(data['Bid']+data['Ask'])/2
            data['Spread']=data['Ask']-data['Bid']
            return data
        except:
            time.sleep(0.2)
    return

def resume_book():
    data=pd.read_csv(dynamic_bookpath(),parse_dates=True)
    book={'Time':[],'id':[],'Pair':[],'Side':[],'Qty':[], 'Trail':[],'TP':[],'SL':[],'PL':[],'OLTP':[],'CLTP':[],'CTime':[]}
    for key in list(data.keys()):
        for i in range(len(data[key])):
            try:
                book[key].append(dt.datetime.fromisoformat(data[key][i]))
            except:
                book[key].append(data[key][i])
    return book


def current_pl(data,pair=None):
    if not pair == None:
        data=data[data['Pair']==pair]
        data=data.reset_index(drop=True)
    data.Time=pd.to_datetime(data.Time)
    data.index=data.Time
    data= data[dt.date.today():]
    data=data.reset_index(drop=True)
    data
    pl=0
    for profit in data.PL:
        if not profit == 'False':
            pl=pl+float(profit)
    openOrder=data
    openOrder=data[data['CLTP']==False]
    try:
        openOrderx=data[data['CLTP']=='False']
    except:
        openOrderx=data[data['CLTP']==False]
    if len(openOrderx) >len(openOrder):
        openOrder=openOrderx
    for i in openOrder.index:
        if openOrder.Side[i] == 'Buy':
            pl=pl+(float(rtd(openOrder.Pair[i])['Bid']-float(openOrder.OLTP[i])))*openOrder.Qty[i]
        else:
            pl=pl+(float(openOrder.OLTP[i])-rtd(openOrder.Pair[i])['Ask'])*openOrder.Qty[i]
    return pl

In [38]:
def stop_for_day(offset=0):
    pl=current_pl(pd.DataFrame(book))
    if pl-offset>max_profit_day or offset+pl<max_loss_day:
        return True
    return False

In [39]:
def momentum(data):
    fastk, fastd = talib.STOCHRSI(data.Close, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
    if int((fastk.iloc[-1]))<int(fastd.iloc[-1]) and int(fastd.iloc[-1]) < int(fastd.iloc[-2]):
        return True
    elif int((fastk.iloc[-1]))>int(fastd.iloc[-1]) and int(fastd.iloc[-1]) > int(fastd.iloc[-2]):
        return False
    return 1

In [40]:
def tp_sl_calculation(data,cdl=10):
    data['BSize']=abs(data.Open-data.Close)
    data['HLSize']=abs(data.High-data.Low)
    data['OHSize']=abs(data.High-data.Open)
    data['OLSize']=abs(data.Open-data.Low)
    bc_avg = (np.average(data.BSize[-cdl:])+min(data.BSize[-cdl:]))/2
    sell  = bc_avg+np.average(data.OLSize[-cdl:])
    low_min = min(data.OLSize[-cdl:])
    low_max = max(data.OLSize[-cdl:])
    high_max = max(data.OHSize[-cdl:])
    high_min = min(data.OHSize[-cdl:])
    buy = bc_avg+np.average(data.OHSize[-cdl:])
    data = data.dropna()
    return [buy,sell,low_min,high_min,low_max,high_max]



def signal(data,ltp,maximum=False):
    ltp = ltp['ltp']
    buy,sell,low_min,high_min,low_max,high_max = tp_sl_calculation(data,cdl=10)
    buy=buy-max_spread_percent*ltp
    sell=sell-max_spread_percent*ltp
    open_price=data.Open[-1]
    high_price=data.High[-1]
    low_price=data.Low[-1]
    open_prev=data.Open[-2]
    high_prev=data.High[-2]
    low_prev=data.Low[-2]
    close_prev=data.Close[-2]
    mid_prev = (open_prev+close_prev)/2
#     print(mid_prev,ltp)
    only = None
    if maximum == True:
        if buy>sell:
            maximum='Buy'
        else:
            maximum='Sell'
    macd, macdsignal, macdhist = talib.MACD(data.Close.iloc[:-1], fastperiod=12, slowperiod=26, signalperiod=9)
    if (only==None or only == 'Buy') and (ltp < (mid_prev)) and (macd.iloc[-2]<macdsignal.iloc[-2],macdhist.iloc[-2]>macdhist.iloc[-3]):# and (high_price < ltp+buy):
        return [True,True,buy,low_max]
    if (only==None or only == 'Sell') and (ltp > (mid_prev)) and (macd.iloc[-2]>macdsignal.iloc[-2],macdhist.iloc[-2]<macdhist.iloc[-3]):# and (low_price > ltp-sell):
        return [True,False,sell,high_max]
    return [False,False,False,False]

In [41]:
def place(pair,side,tp,sl,qty=1,offset=0):
    global book
    try:
        book=resume_book()
        pass
    except:
        pass
    openOrdert=pd.DataFrame(book)
    if trade_stop:
        print('day pl limit reached')
        return False    
    if len(openOrdert)>=maxtrade:
        #print('Max trades already exist')
        return False
    try:
        openOrderx=openOrdert[openOrdert['CLTP']=='False']
    except:
        openOrderx=openOrdert[openOrdert['CLTP']==False]
    openOrder=openOrdert[openOrdert['CLTP']==False]
    if len(openOrderx)>len(openOrder):
        openOrder=openOrderx
    if len(openOrder)>=max_concurrent_trade:
        #print('Many Trades running')
        return
    try:
        openOrder=openOrder[openOrder['Pair']==pair]
        if len(openOrder)>=max_open_trade_per_pair:
            return False
    except:
        pass
    #########################################################
    if side=='Buy':
        ltp=rtd(pair)['Ask']
    else:
        ltp=rtd(pair)['Bid']
    if demo==False:
        if not is_mis == True:
            pass
        else:
            pass
        if not response['status'] == 'success':
            print(response)
            return 
        order_id=response['data']['oms_order_id']
    else:
        try:
            order_id=max(book['id'])+1
        except:
            order_id=1
    
    #########################################################
    book['id'].append(order_id)
    book['Time'].append(dt.datetime.now())
    book['Pair'].append(pair)
    book['Qty'].append(qty)
    book['Side'].append(side)
    book['Trail'].append(False)
    book['PL'].append(False)
    book['OLTP'].append(float(ltp)+offset)
    book['CLTP'].append(False)
    book['CTime'].append(False)
    book['TP'].append(ltp+float(tp) if side=='Buy' else ltp-float(tp))
    tp=ltp+float(tp) if side=='Buy' else ltp-float(tp)
    book['SL'].append(ltp-float(sl) if side=='Buy' else ltp+float(sl))
    pd.DataFrame(book).to_csv(dynamic_bookpath(),index=False)
    msg = 'BOT1 '+'id: '+str(order_id)+' '+side+' '+pair+' '+' Price: '+str(ltp)+' Qty: '+str(qty)+' TP:'+str(tp)
    telegram(msg)  


In [42]:
def closeTrade(tk):
    global book
    try:
        book=resume_book()
        pass
    except:
        pass
    openOrder=pd.DataFrame(book)
    index=tk-1
#     openOrder=openOrder[openOrder['id']==int(tk)]
    
    side=book['Side'][index]
    pair=book['Pair'][index]
    cltp=book['CLTP'][index]
    qty=int(book['Qty'][index])
    if not (cltp == False or cltp == 'False'):
        #print('Trade already closed')
        return
    if side=='Buy':
        ltp=rtd(pair)['Bid']
        book['PL'][index]=(ltp-float(book['OLTP'][index]))*qty
        pl=(ltp-float(book['OLTP'][index]))*qty
    else:
        ltp=rtd(pair)['Ask']
        book['PL'][index]=(float(book['OLTP'][index])-ltp)*qty
        pl=(float(book['OLTP'][index])-ltp)*qty
    
    book['CLTP'][index]=ltp
    book['CTime'][index]=dt.datetime.now()
    pd.DataFrame(book).to_csv(dynamic_bookpath(),index=False)
    telegram('BOT1: Trade closed '+str(side)+' '+str(pair)+' Price: '+str(ltp)+' Quantity: '+str(qty)+' Profit/Loss: '+str(pl))
    try:
        book=resume_book()
        pass
    except:
        pass


In [43]:

def closeAll(pair=None):
    global book
    try:
        resume_book()
        openOrderx=pd.read_csv(dynamic_bookpath(),parse_dates=True)
    except:
        openOrderx=pd.DataFrame(book)
    openOrder=openOrderx[openOrderx['PL'] == False]
    try:
        openOrdert=openOrderx[openOrderx['PL'] == 'False']
        if len(openOrder)<len(openOrdert):
            openOrder=openOrdert
    except:
        pass
    if not pair==None:
        openOrder=openOrder[openOrder['Pair']==pair]
    for i in openOrder.id:
        closeTrade(i)
    try:
        book=resume_book()
    except:
        pass

def check_suspension(pair='EUR/USD'):
    global book
    try:
        book=resume_book()
        pass
    except:
        pass
    data=pd.DataFrame(book)
    if len(data)==0:
        return True
    data_temp=data#data[data['PL']!=False].reset_index(drop=True)
    data_temp=data_temp[data_temp['Pair']==pair].reset_index(drop=True)
    pl=current_pl(data,pair)
#     print(pl)
    if pl < suspension_limit:
        last_placed=pd.to_datetime(data_temp.Time).iloc[-1]
        if last_placed+dt.timedelta(seconds=suspnsion_time) > dt.datetime.now():
            closeAll(pair)
            stopped_pair[pair]=True
            return False
    return True

def check_daily_limit():
    global book
    try:
        resume_book()
        data=pd.read_csv(dynamic_bookpath(),parse_dates=True)
    except:
        data=pd.DataFrame(book)
    if len(data)==0:
        return
    for pair in pairs:
#         print(current_pl(data,pair))
        if current_pl(data,pair)>max_profit_per_day_per_pair:
            stopped_pair[pair]=True
            #closeAll(pair)
    if current_pl(data,pair)<max_profit_per_day_per_pair:
        stopped_pair[pair]=False


In [44]:
def checkClose():
    global book
    try:
        book=resume_book()
        openOrdert=pd.read_csv(dynamic_bookpath(),parse_dates=True)
    except:
        openOrdert=pd.DataFrame(book)
    try:
        openOrderx=openOrdert[openOrdert['CLTP']=='False']
    except:
        openOrderx=openOrdert[openOrdert['CLTP']==False]
    openOrder=openOrdert[openOrdert['CLTP']==False]
    if len(openOrderx)>len(openOrder):
        openOrder=openOrderx
    index=list(openOrder.index)
    if len(index)==0:
        return
    for i in index:
        tp = float(book['TP'][i])
        sl = float(book['SL'][i])
        side = book['Side'][i]
        otime = book['Time'][i]
        pair = book['Pair'][i]
        oltp = float(book['OLTP'][i])
        tk = book['id'][i]
        if side=='Buy':
            ltp=rtd(pair)['Bid']
        else:
            ltp=rtd(pair)['Ask']
        try:
            if type(otime)==str:
                otime=dt.datetime.strptime(otime, '%Y-%m-%d %H:%M:%S.%f')
        except:
            pass
        if dt.timedelta(seconds=trade_duration)+otime <= dt.datetime.now():
            print('Time Close')
            closeTrade(tk)
            
        high=max(tp,sl)
        low=min(tp,sl)
        if side=='Buy':
            if pair=='USD/JPY':
                pl=(ltp-oltp)*10
            else:
                pl=(ltp-oltp)*1000
        else:
            if pair=='USD/JPY':
                pl=(oltp-ltp)*10
            else:
                pl=(oltp-ltp)*1000
        if (low >= ltp or high <= ltp or pl<max_loss_per_trade):
            closeTrade(tk)

In [45]:
def checkPlace():
    global book
    pass_trade=False
    side_restricted=None
    try:
        book=resume_book()
        openOrder=pd.read_csv(dynamic_bookpath(),parse_dates=True)
    except:
        openOrder=pd.DataFrame(book)
    for pair in pairs:
        time_now=dt.datetime.now()#+dt.timedelta(seconds=waiting_time)
        time_now=time_now.replace(minute=0,second=0,microsecond=0)
        data=ohlc(pair)
        try:
            last_placed=openOrder[openOrder['Pair']==pair]
        except:
            last_placed=[]
            
        if len(last_placed)>0:
            wait_placed=last_placed.reset_index(drop=True)
            wait_temp_restrictedy=[]
            wait_temp_restricted=[]
            try:
                wait_temp_restricted=wait_placed.drop(wait_placed[wait_placed['PL'] == 'False'].index)
            except:
                pass     
            try:
                wait_temp_restricted=wait_temp_restricted.reset_index(drop=True)
                wait_temp_restricted.Time=pd.to_datetime(wait_temp_restricted.Time)
                wait_placed_time=wait_temp_restricted.Time.iloc[-1]
                wait_placed_time=wait_placed_time.replace(minute=0,second=0,microsecond=0)
                wait_temp_restricted.PL=pd.to_numeric(wait_temp_restricted.PL)
                wait_placed_pl=wait_temp_restricted.PL.iloc[-1]
                if wait_placed_pl<0:
                    if wait_placed_time+dt.timedelta(seconds=waiting_time)>time_now:
                        continue
                else:
                    pass
            except:
                pass
            
        time_now=dt.datetime.now()+dt.timedelta(seconds=waiting_time)
        time_now=time_now.replace(minute=0,second=0,microsecond=0)
        
        if len(last_placed)>0:
            last_placed=last_placed.reset_index(drop=True)
            temp_restricted=last_placed[last_placed['CLTP']==False]
            temp_restricted=temp_restricted.reset_index(drop=True)
            if len(temp_restricted)>=max_concurrent_pair:
                pass_trade=True
            if len(temp_restricted)>0:
                side_restricted = 'Sell' if temp_restricted.Side.iloc[-1]=='Buy' else 'Buy'
            last_placed=last_placed.Time.iloc[-1]
            try:
                if type(last_placed)==str:
                    last_placed=dt.datetime.strptime(last_placed, '%Y-%m-%d %H:%M:%S.%f')
            except:
                pass
            last_placed=last_placed.replace(minute=0,second=0,microsecond=0)
            if last_placed==time_now or last_placed >= time_now:
                last_placed=False
        else:
            last_placed=True
        if not last_placed==False:
            last_placed=True
        now = signal(ohlc(pair),rtd(pair))
        if last_placed and rtd(pair)['Spread']/(rtd(pair)['ltp'])<max_spread_percent and now[0] == True and pass_trade == False and check_suspension(pair) and stopped_pair[pair]==False:
            if now[1] == True and (not side_restricted == 'Buy'):
                #print('Buy Trade place')
                tp=now[2]
                sl=now[3]*20
                if pair == 'USD/JPY':
                    place(pair,'Buy',tp,sl,qty=10)
                else:
                    place(pair,'Buy',tp,sl,qty=1000)
            elif (not side_restricted == 'Sell'):
                #print('Sell trade Placed')
                tp=now[2]
                sl=now[3]*20
                if pair == 'USD/JPY':
                    place(pair,'Sell',tp,sl,qty=10)
                else:
                    place(pair,'Sell',tp,sl,qty=1000)

In [46]:
try:
    book=resume_book()
    pass
except:
    pass
pd.DataFrame(book)

,Time,id,Pair,Side,Qty,Trail,TP,SL,PL,OLTP,CLTP,CTime
0,2020-09-25 15:43:57.854034,1,EUR/USD,Buy,1000,False,1.163409,1.10557,1.8600000000001948,1.16157,1.1634300000000002,2020-09-25 19:20:26.575245
1,2020-09-25 15:43:58.083318,2,GBP/USD,Buy,1000,False,1.272503,1.12196,3.0600000000000627,1.26946,1.27252,2020-09-25 18:22:33.056992
2,2020-09-25 15:43:58.233260,3,USD/CHF,Sell,1000,False,0.928442,0.98429,1.0499999999998844,0.92949,0.9284399999999999,2020-09-25 19:20:26.620687
3,2020-09-28 06:36:03.887046,4,EUR/USD,Buy,1000,False,1.164743,1.10770,1.480000000000148,1.16340,1.16488,2020-09-28 09:29:57.919376
4,2020-09-28 06:36:04.417469,5,GBP/USD,Sell,1000,False,1.274460,1.38248,-5.009999999999959,1.27698,1.28199,2020-09-28 08:31:20.704203
5,2020-09-28 06:36:04.643721,6,USD/CHF,Sell,1000,False,0.928210,0.97512,0.940000000000052,0.92902,0.9280799999999999,2020-09-28 08:03:38.150802
6,2020-09-28 06:50:55.090651,7,AUD/USD,Buy,1000,False,0.707047,0.61359,1.2600000000000389,0.70519,0.70645,2020-09-28 10:50:55.360972
7,2020-09-28 08:03:38.914350,8,USD/CHF,Buy,1000,False,0.929082,0.89119,-1.6399999999999748,0.92809,0.92645,2020-09-28 12:03:39.282270
8,2020-09-28 09:29:58.249410,9,EUR/USD,Sell,1000,False,1.163406,1.20664,-2.2200000000001108,1.16484,1.16706,2020-09-28 13:29:58.603766
9,2020-09-28 11:08:18.645145,10,AUD/USD,Sell,1000,False,0.704105,0.75407,0.6099999999999994,0.70627,0.7056600000000001,2020-09-28 15:08:19.144977


In [47]:
# while True:
#     while tradeTime():
#         checkPlace()
#         checkClose() 
#         check_daily_limit()
        

In [ ]:
while True:
    while tradeTime():
        try:
            if tradeTime(offset=True,starttime=500,endtime=2000) and trade_stop==False:
                checkPlace()
            checkClose() 
            check_daily_limit()
        except Exception as e:
            print(str(e))
            f = open("log.txt", "a")
            f.write(str(e)+"\n")
            f.close()
            time.sleep(1)
            pass
        
        if stop_for_day():
            trade_stop=True
            print('stop for day close all')
            stopped_pair={'EUR/USD':True, 'GBP/USD':True, 'USD/CHF':True,'AUD/USD':True}
            closeAll()
        
        if tradeTime(offset=True,starttime=500,endtime=530):
            trade_stop=False
            stopped_pair={'EUR/USD':False, 'GBP/USD':False, 'USD/CHF':False,'AUD/USD':False}
        
        

            
#     if not tradeTime(offset=True,starttime=0,endtime=1):
#         trade_stop=False
#         stopped_pair={'EUR/USD':False, 'GBP/USD':False, 'USD/CHF':False,'AUD/USD':False}
        
#     if not tradeTime(offset=True,starttime=500,endtime=630):
#         trade_stop=False
#         stopped_pair={'EUR/USD':False, 'GBP/USD':False, 'USD/CHF':False,'AUD/USD':False}

#     if not tradeTime():
#         closeAll()
#         stopped_pair={'EUR/USD':False, 'GBP/USD':False, 'USD/CHF':False,'AUD/USD':False}
#         trade_stop=False

Time Close
BOT1: Trade closed Sell GBP/USD Price: 1.2962099999999999 Quantity: 1000 Profit/Loss: -0.8999999999996788
BOT1: Trade closed Sell EUR/USD Price: 1.17794 Quantity: 1000 Profit/Loss: -2.749999999999808


In [ ]:
current_pl(pd.DataFrame(book),'EUR/USD')

In [ ]:
tradeTime()

In [ ]:
while True:
    while tradeTime():
        checkPlace()
        checkClose() 
        check_daily_limit()
        

In [ ]:
stopped_pair={'EUR/USD':False, 'GBP/USD':False, 'USD/CHF':False,'AUD/USD':False}


In [ ]:
stopped_pair

In [ ]:
trade_stop=False
stopped_pair={'EUR/USD':False, 'GBP/USD':False, 'USD/CHF':False,'AUD/USD':False}


In [ ]:
current_pl(pd.DataFrame(book))